# Exercício 2:

Neste exercício foi nos pedido que implementassemos um algoritmo de codificação com um modo semelhante a uma cifra *one-time pad*. Por outras palavras, é esperado que seja desenvolvida que dada uma "chave" deverá realizar a operação XOR entre a mensagem que se pretende partilhar e a "chave".

Esta cifra deve apresentar um comportamento semelhante ao das cifras em bloco, dado que a mensagem deverá ser dividida em blocos de 64 bits, e cada um destes blocos deverá ser cifrado com um bloco da "chave".

Esta "chave" será obtida utilizando um gerador definido recorrendo à função hash SHAKE-256. Este gerador terá uma seed definida em função de uma password. 

**Deverá ser também implementada uma autenticação de metadados.**
**Aparte: Não deveremos usar a mesma chave para cada bocado da mensagem. Como tipicamente, algoritmos criptográficos estão disponiveis para qualquer pessoa, um atacante pode descobrir a chave que estou a utilizar num algoritmo one-time pad se este for utilizada muitas vezes. Portanto, devemos associar a cada bloco do plaintext um bloco de output diferente. **

Adicionalmente, esta cifra deverá implementar uma autenticação de metadados. Para tal, iremos utilizar....

In [1]:
## Imports:
import os
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.hashes import SHAKE256
from cryptography.hazmat.primitives import hashes, padding

## Gerador de uma seed:

Para esta implementação, como já foi mencionado para a nossa implementação, iremos obter uma seed que será aplicado ao gerador para obter a "chave".

Para obtermos esta seed, utilizaremos um algoritmo KDF (key derivation function). Ao algoritmo KDF será introduzida uma password escolhida pelo utilizador. PAra além disto, este algoritmo deverá receber um length e um salt. O salt será um número random, **e o length será 8**.

A seed gerada será a nossa **cipher_key**.

Utilizamos o PBKDF2HMAC porque .....

In [2]:
def generateSeed(pwd):
    salt = os.urandom(16)
    length = 32 # Length in bytes das chaves derivadas
    
    kdf = PBKDF2HMAC(algorithm=hashes.SHA256(), 
                      length=length,
                      salt=salt,
                      iterations = 480000,
                     )
     
    seed = kdf.derive(pwd)
    
    return seed

## Gerador Pseudo-aleatório:

Como já referimos, será necessário utilizar um gerador pseudo-aleatório para obter uma chave que será utilizado na codificação/descodificação da mensagem. Para tal, iremos utilizar a função hash SHAKE-256. A esta função será passada a cipher_key (seed). Isto será crucial, uma vez que será necessário que ambos os participantes possam obter a mesma chave uma vez que se a chave for diferente estes poderão não ser capazes de descodificar a mensagem.

Este gerador deverá gerar 2^n palavras, tal como foi especificado no enunciado. **Dado que um bloco terá o tamanho de 64 bits (8bytes), cada palavra deverá ter o mesmo tamanho que um bloco, portanto, precisamos de um gerador cujo output é 2^n * 8 bytes. **

In [3]:
# Function that creates a generator with max 2^n words of 8 bytes.
def generator(n, seed):
    digest = hashes.Hash(hashes.SHAKE256(2**n*8))
    
    digest.update(seed) # Define seed
    
    gen = digest.finalize()
    return gen

## Cifrar:

De seguida iremos implementar a funcionalida de cifrar. Para tal será necessário, gerar a "chave" utilizando o gerador pseudo-aleatório desenvolvido anteriormente, após obter uma seed em função de uma password.

A mensagem a cifrar e a "chave" a utilizar deverão ser partidas em blocos e a cada bloco será aplicada a operação XOR.

Sabendo que em python um char corresponde a um byte, e que 64 bits equivale a 8 bytes, devemos dividir uma mensagem em blocos de 8 chars.

**Explicar Padding**

**Juntamente com a mensagem foram também enviados metadados, de forma a que a cifra possua também autenticação de meta-dados**

In [4]:
def pad(last_block, size_block):
    len_block = len(last_block)
    padding_val = size_block - len_block
    
    #Size of padding in bits
    padder = padding.PKCS7(padding_val*8).padder()
    last_block = bytes(last_block,'utf-8')
    
    last_block = padder.update(last_block)
    
    last_block += padder.finalize()
    
    last_block = last_block.decode("utf-8")
    
    return last_block, padding_val


def encrypt(n, pwd, msg):    
    
    # Checks the value of n, and if the msg size is valid?
    
    NBytes = 8
    
    # generates a seed based on a password
    cipher_key = generateSeed(bytes(pwd, 'utf-8'))
    
    # obtains the output of a pseudo-random generator based on a cipher_key
    output = generator(n, cipher_key)
    
    #Break message into chunks of 8 bytes
    block_plaintext = [msg[i:i+NBytes] for i in range(0, len(msg), NBytes)]    
    
    mod = len(msg) % NBytes

    if(mod > 0):
        coef = len(msg) // NBytes
        padded_block, padded_size = pad(block_plaintext[coef], NBytes)
        block_plaintext[coef] = padded_block
    else:
        padded_size = 0
    
    #Breaks the output of the generator into blocks of 8 bytes
    block_output = [output[i:i+NBytes] for i in range(0, len(output), NBytes)]     
  
    # Aplies the XOR operation to the blocks
    cipher = opXOR(block_plaintext, block_output)
    
    print("CIPHERTEXT: "+cipher)
        
    # Obter Meta dados
    metadata = os.urandom(16) # Temporário
    pkg = {'ciphertext': cipher,'cipher_key': cipher_key, 'padsize': padded_size, 'metadata': metadata}
    
    return pkg

    # encriptar metadados?

Utilizaremos esta função auxiliar tanto para decifrar como para cifrar, dado que esta permite aplicar a operação XOR a dois conjuntos de blocos:

In [5]:
# msg, key are sepated in blocks
def opXOR(msg, key):
    output = ""
    for msg_block, key_block in zip(msg, key):
        for i, j in zip(msg_block, key_block):
            word = ord(i) ^ j # Xor needs to be used between chars
            output += chr(word) # Guardar esta word
    return output

## Decifrar:

De forma a completar o modo da cifra é necessário implementar o método de decifrar o texto cifrado recebido. Para que esta descodificação seja possível, é necessário que o participante que pertende executar essa funcionalidade seja capaz de gerar a "chave" que foi usada para codificar a mensagem. Assim, este deverá receber a **cipher_key** usada para encriptar o plaintext.

Com a **cipher_key**, deverá ser obtido o resultado de um gerador pseudo-aleatório. Com este resultado, iremos executar uma operação XOR a blocos de 64 bits do ciphertext e a 64 bits do output do gerador.

In [6]:
def unpad(last_block, size_padding):
    
    unpadder = padding.PKCS7(size_padding*8).unpadder()
    last_block = bytes(last_block,'utf-8')
    
    last_block = unpadder.update(last_block)
    last_block += unpadder.finalize()
    
    last_block = last_block.decode("utf-8")
    
    return last_block
    

def decrypt(pwd, msg, n):
    
    NBytes = 8
    
    ct = msg['ciphertext']
    cipher_key = msg['cipher_key']
    meta = msg['metadata']
    padded_size = msg['padsize']
   
    output = generator(8, cipher_key)
    
    block_ciphertext = [ct[i:i+NBytes] for i in range(0, len(ct), NBytes)]        
    block_output = [output[i:i+NBytes] for i in range(0, len(output), NBytes)]  
    
    if(padded_size > 0):
        plaintext = opXOR(block_ciphertext[:-1], block_output[:-1])
        coef = len(ct) // NBytes
        last_cipherblock = block_ciphertext[coef]
        last_outputblock = block_output[coef]
        last_plainblock = opXOR([last_cipherblock], [last_outputblock])
        unpadded_block = unpad(last_plainblock, padded_size)
        plaintext += unpadded_block
    
    else:
        plaintext = opXOR(block_ciphertext, block_output)    
    
    print("PLAINTEXT: "+plaintext)

## Classe de teste:

Por último, definimos uma função de teste, onde teremos uma chamada da função de cifragem e uma chamada à função de decifragem, permitindo-nos testar se os métodos se encontram corretamente desenvolvidos.

In [7]:
def teste (pwd, n, msg):
    
    ciphertext = encrypt (n, pwd, msg)
    
    decrypt (pwd, ciphertext, n)
    
    

In [8]:
teste ("password", 10, "Esta mensagem é apenas um teste para verificar o funcionamento do programa.")

CIPHERTEXT: 4Yí:.JÇGª?}±âHÂjpÕ¶}³37ìmuËî0¬ v¢Jñ	,E58>uÙÓuµ_>@Ê]vHvr	¼
PLAINTEXT: Esta mensagem é apenas um teste para verificar o funcionamento do programa.
